In [15]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.models import WDL
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names

data = pd.read_csv('movielens_sample.txt')
sparse_features = ['user_id', 'movie_id', 'timestamp', 'title', 'genres', 'gender', 'occupation', 'zip']
dense_features = ['age']
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique())
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1)
                      for feat in dense_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'] )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=True, validation_split=0.2 )
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)


Train on 128 samples, validate on 32 samples
Epoch 1/10


C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
C:\Users\TAOXUEJIE-PSD\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


128/128 [==============================] - 2s 14ms/sample - loss: 9.6857 - mse: 9.6857 - val_loss: 8.3633 - val_mse: 8.3633
Epoch 2/10
128/128 [==============================] - 0s 156us/sample - loss: 9.4223 - mse: 9.4222 - val_loss: 8.1476 - val_mse: 8.1475
Epoch 3/10
128/128 [==============================] - 0s 171us/sample - loss: 9.1681 - mse: 9.1681 - val_loss: 7.9370 - val_mse: 7.9370
Epoch 4/10
128/128 [==============================] - 0s 171us/sample - loss: 8.9198 - mse: 8.9198 - val_loss: 7.7286 - val_mse: 7.7286
Epoch 5/10
128/128 [==============================] - 0s 172us/sample - loss: 8.6719 - mse: 8.6719 - val_loss: 7.5217 - val_mse: 7.5217
Epoch 6/10
128/128 [==============================] - 0s 164us/sample - loss: 8.4245 - mse: 8.4244 - val_loss: 7.3165 - val_mse: 7.3165
Epoch 7/10
128/128 [==============================] - 0s 164us/sample - loss: 8.1784 - mse: 8.1784 - val_loss: 7.1139 - val_mse: 7.1139
Epoch 8/10
128/128 [==============================] - 0s 140